# Excess Mortality And Vaccines In Europe (v2)

**Author:** Justin Garza

**Date:** See below  
  
**Description:**  
This notebook explores excess mortality across Europe, analyzing statistical trends and investigating potential causes through data visualization and interpretation.  

**Content Warning:**    
If you find discussions of death or its underlying factors distressing, please proceed with caution or consider whether this content is right for you.  

In [455]:
from datetime import datetime
from IPython.display import display
from IPython.display import Markdown as MD
current_date = datetime.now().strftime('%Y-%m-%d')
version = datetime.now().strftime('%Y%m%d.%H%M')
display(MD(f"**Date:** {current_date}"))
display(MD(f"**version:** {version}"))

**Date:** 2025-02-05

**version:** 20250205.2209

## Table of Contents

In [456]:
import os
import json5 as json
from IPython.display import display, HTML
from IPython.display import Markdown as MD

nb_path = os.path.join(os.getcwd(),'main.ipynb')

# Read the notebook
with open(nb_path, 'r', encoding='utf-8') as f:
    nb_data = json.load(f)

# Extract headers from markdown cells
outline = []
for cell in nb_data.get("cells", []):
    if cell["cell_type"] == "markdown":
        for line in cell["source"]:
            if line.startswith("#"):  # Markdown header
                level = line.count("#")  # Determine header level
                title = line.lstrip("#").strip()
                outline.append({
                        'text':title,
                        'level':level-1,
                        'link': f"index.html#{title.replace(' ','-')}"
                        })

for i in outline:
    display(HTML(f'<a href=\'{i["link"]}\'>{"____" * i["level"]}{i["text"]}</a>'))



## Prerequisites

### Scientific Method 
The **scientific method** is a systematic approach to investigating natural phenomena, acquiring knowledge, and testing hypotheses. It consists of the following key steps:

1. **Observation**  
   - Identify a problem or phenomenon that needs explanation.
   - Gather initial data through direct observation or research.

2. **Hypothesis**  
   - Propose a testable and falsifiable explanation (a hypothesis).
   - Example: "If plants receive more sunlight, then they will grow taller."

3. **Experimentation**  
   - Design and conduct controlled experiments to test the hypothesis.
   - Include independent and dependent variables, control groups, and repeatable procedures.

4. **Conclusion**  
   - Determine whether the data supports or refutes the hypothesis.
   - Modify or refine the hypothesis if necessary.

5. **Replication & Peer Review**  
   - Repeat experiments to verify results.
   - Publish findings for scrutiny by the scientific community.

The scientific method ensures **objectivity, reliability, and accuracy** in scientific inquiry. It is an iterative process, meaning that conclusions can lead to new questions and further investigations.


### Logical Fallacies
Logical fallacies are errors in reasoning that weaken arguments. They can be categorized into **formal** (structural errors) and **informal** (content errors).


| **Type**                 | **Fallacy**                    | **Description** |
|--------------------------|--------------------------------|----------------|
| Formal                   | Affirming the Consequent      | Assuming that if *P → Q* and *Q is true*, then *P must be true*. |
|                          | Denying the Antecedent        | Assuming that if *P → Q* and *P is false*, then *Q must be false*. |
|                          | Non-Sequitur                  | The conclusion does not logically follow from the premises. |
| Informal – Relevance     | **Ad Hominem**                    | Attacking the person instead of the argument. |
|                          | **Straw Man**                     | Misrepresenting an argument to make it easier to attack. |
|                          | **Red Herring**                   | Diverting attention with an irrelevant point. |
|                          | **Appeal to Authority**           | Claiming something is true because an authority said so. |
|                          | **Appeal to Emotion**             | Using emotions instead of logic to argue a point. |
| Informal – Causation & Presumption | Post Hoc Ergo Propter Hoc | Assuming that correlation implies causation. |
|                          | Slippery Slope                | Claiming one action will lead to extreme consequences. |
|                          | False Dilemma                 | Presenting only two options when more exist. |
|                          | Begging the Question          | Using circular reasoning. |
|                          | False Equivalence             | Treating two things as equal when they are not. |
|                          | Hasty Generalization          | Drawing a conclusion from insufficient evidence. |
|                          | No True Scotsman              | Excluding counterexamples by redefining a group. |

Logical fallacies can make arguments misleading or invalid. Identifying them helps improve critical thinking and debate skills.




## Setup
In this section, we prepare the notebook by importing necessary libraries, configuring settings, and setting up directories for data and outputs. The setup ensures the environment is ready for data analysis and visualization.


In [457]:
# this code to will import all the things i need for this notebook

import os
import re
import math

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML
from IPython.display import Markdown as MD


# Graphs and Charts
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns
import plotly.express as px
# use to export plotly graphs
import plotly.io as pio 

#misc
from scipy.stats import spearmanr, kendalltau
import pycountry

# pandas Settings/Options
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 9000)
pd.set_option('max_colwidth', 400)

# colormap 
heatmapCM = sns.color_palette('Spectral_r', as_cmap=True)


## directories 
DIR = os.getcwd()
print(f'{DIR=}')

DataDIR = os.path.join(DIR,'data')
OutDIR = os.path.join(DIR,'docs')

if not os.path.exists(DataDIR):
    print('***DATA FOLDER IS MISSING***')

if not os.path.exists(OutDIR):
    os.makedirs(OutDIR)


DIR='c:\\Users\\JGarza\\GitHub\\Excess_Mortality_And_Vaccines_In_Europe'


## Helping Functions
This section defines utility functions that streamline repetitive tasks and improve code readability. These functions will be used throughout the notebook to simplify operations, enhance modularity, and reduce redundancy.

In [458]:
def df_column_uniquify(df):
    '''
    renames columns that are the same
    '''
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [459]:
def abbr_to_isoalpha3(abbr):
    """
    Convert a European country ISO Alpha-2 code to ISO Alpha-3 code.

    Parameters:
        abbreviation (str): ISO Alpha-2 country code (e.g., 'DE' for Germany).

    Returns:
        str: ISO Alpha-3 country code (e.g., 'DEU'), or None if not found.
    """
    try:
        country = pycountry.countries.get(alpha_2=abbr.upper())
        if country:
            return country.alpha_3
        else:
            return None
    except KeyError:
        return None

In [460]:
# this if for converting between the abbreviation andand the names of the countries

country_dict = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "DE": "Germany",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
    "IS": "Iceland",
    "LI": "Liechtenstein",
    "NO": "Norway",
    "CH": "Switzerland",
    "UK": "United Kingdom",
    "ME": "Montenegro",
    "GE": "Georgia",
    "AL": "Albania",
    "RS": "Serbia",
    "AD": "Andorra",
    "AM": "Armenia"
}

def abbr_to_name(abbreviation):
    return country_dict.get(abbreviation.upper(), "Abbreviation not found")

def name_to_abbr(name):
    reverse_dict = {v: k for k, v in country_dict.items()}
    return reverse_dict.get(name, "Unknown")

# testing
print( abbr_to_name("BE") )  # Output: 'Belgium'
print( name_to_abbr("Belgium") )  # Output: 'BE'

Belgium
BE


## Import and Clean Data

### death data

#### Getting the Data
1. go to [Europa.eu - Database](https://ec.europa.eu/eurostat/databrowser/explore/all/popul)
2. choose 
    * Population and social conditions 
    * Demography, population stock and balance 
    * Deaths by week – special data collection
    * Deaths by week, sex and 20-year age group
3. Click the little table 󰓫
4. customize the data 
    * Customize your dataset -> Time -> From - to 
        * From: 2015-W01
        * To: [Current or Max]
    * Move the `Age Class` under `Geopolitical entity (reporting)`
5. Click `download` (as a spreadsheet) and place the file in the `.\data` folder


<img src='./docs/chrome_kwaSkg7mq6.png' height=200/>

#### variables
* dd = death data
* ddy = death data (by year)
* ddn = normalized deaths (by year)

In [461]:
# getting the data 
dd = pd.read_excel(os.path.join(DataDIR,"demo_r_mwk_20__custom_15220765_page_spreadsheet.xlsx"),sheet_name = "Sheet 1")

# remove the headers
dd = dd.iloc[7::]

# drop the bad columns
for c in dd.columns:
    if pd.isnull(dd.at[7,c]):
        dd = dd.drop(columns=[c])

# rename time columns
for c in dd.columns:
    name = dd.at[7,c]
    dd = dd.rename(columns={c: name})

# make the duplicate column names unique
dd = df_column_uniquify(dd)

# # rename the first two columns
dd = dd.rename(columns={'TIME': 'abbr'})
dd = dd.rename(columns={'TIME_1':'name'})
dd = dd.rename(columns={'TIME_2':'agegrp'})
dd = dd.rename(columns={'TIME_3':'agegrp_desc'})
           
# drop, replace, reset index,
dd = dd.drop([7,8,9])
dd = dd.replace(to_replace=':', value=None)
dd = dd.reset_index(drop=True)

# display(dd.head(3))
display(dd.tail(10))

c:\Users\JGarza\pythons\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,abbr,name,agegrp,agegrp_desc,2015-W01,2015-W02,2015-W03,2015-W04,2015-W05,2015-W06,2015-W07,2015-W08,2015-W09,2015-W10,2015-W11,2015-W12,2015-W13,2015-W14,2015-W15,2015-W16,2015-W17,2015-W18,2015-W19,2015-W20,2015-W21,2015-W22,2015-W23,2015-W24,2015-W25,2015-W26,2015-W27,2015-W28,2015-W29,2015-W30,2015-W31,2015-W32,2015-W33,2015-W34,2015-W35,2015-W36,2015-W37,2015-W38,2015-W39,2015-W40,2015-W41,2015-W42,2015-W43,2015-W44,2015-W45,2015-W46,2015-W47,2015-W48,2015-W49,2015-W50,2015-W51,2015-W52,2015-W53,2016-W01,2016-W02,2016-W03,2016-W04,2016-W05,2016-W06,2016-W07,2016-W08,2016-W09,2016-W10,2016-W11,2016-W12,2016-W13,2016-W14,2016-W15,2016-W16,2016-W17,2016-W18,2016-W19,2016-W20,2016-W21,2016-W22,2016-W23,2016-W24,2016-W25,2016-W26,2016-W27,2016-W28,2016-W29,2016-W30,2016-W31,2016-W32,2016-W33,2016-W34,2016-W35,2016-W36,2016-W37,2016-W38,2016-W39,2016-W40,2016-W41,2016-W42,2016-W43,2016-W44,2016-W45,2016-W46,2016-W47,2016-W48,2016-W49,2016-W50,2016-W51,2016-W52,2017-W01,2017-W02,2017-W03,2017-W04,2017-W05,2017-W06,2017-W07,2017-W08,2017-W09,2017-W10,2017-W11,2017-W12,2017-W13,2017-W14,2017-W15,2017-W16,2017-W17,2017-W18,2017-W19,2017-W20,2017-W21,2017-W22,2017-W23,2017-W24,2017-W25,2017-W26,2017-W27,2017-W28,2017-W29,2017-W30,2017-W31,2017-W32,2017-W33,2017-W34,2017-W35,2017-W36,2017-W37,2017-W38,2017-W39,2017-W40,2017-W41,2017-W42,2017-W43,2017-W44,2017-W45,2017-W46,2017-W47,2017-W48,2017-W49,2017-W50,2017-W51,2017-W52,2018-W01,2018-W02,2018-W03,2018-W04,2018-W05,2018-W06,2018-W07,2018-W08,2018-W09,2018-W10,2018-W11,2018-W12,2018-W13,2018-W14,2018-W15,2018-W16,2018-W17,2018-W18,2018-W19,2018-W20,2018-W21,2018-W22,2018-W23,2018-W24,2018-W25,2018-W26,2018-W27,2018-W28,2018-W29,2018-W30,2018-W31,2018-W32,2018-W33,2018-W34,2018-W35,2018-W36,2018-W37,2018-W38,2018-W39,2018-W40,2018-W41,2018-W42,2018-W43,2018-W44,2018-W45,2018-W46,2018-W47,2018-W48,2018-W49,2018-W50,2018-W51,2018-W52,2019-W01,2019-W02,2019-W03,2019-W04,2019-W05,2019-W06,2019-W07,2019-W08,2019-W09,2019-W10,2019-W11,2019-W12,2019-W13,2019-W14,2019-W15,2019-W16,2019-W17,2019-W18,2019-W19,2019-W20,2019-W21,2019-W22,2019-W23,2019-W24,2019-W25,2019-W26,2019-W27,2019-W28,2019-W29,2019-W30,2019-W31,2019-W32,2019-W33,2019-W34,2019-W35,2019-W36,2019-W37,2019-W38,2019-W39,2019-W40,2019-W41,2019-W42,2019-W43,2019-W44,2019-W45,2019-W46,2019-W47,2019-W48,2019-W49,2019-W50,2019-W51,2019-W52,2020-W01,2020-W02,2020-W03,2020-W04,2020-W05,2020-W06,2020-W07,2020-W08,2020-W09,2020-W10,2020-W11,2020-W12,2020-W13,2020-W14,2020-W15,2020-W16,2020-W17,2020-W18,2020-W19,2020-W20,2020-W21,2020-W22,2020-W23,2020-W24,2020-W25,2020-W26,2020-W27,2020-W28,2020-W29,2020-W30,2020-W31,2020-W32,2020-W33,2020-W34,2020-W35,2020-W36,2020-W37,2020-W38,2020-W39,2020-W40,2020-W41,2020-W42,2020-W43,2020-W44,2020-W45,2020-W46,2020-W47,2020-W48,2020-W49,2020-W50,2020-W51,2020-W52,2020-W53,2021-W01,2021-W02,2021-W03,2021-W04,2021-W05,2021-W06,2021-W07,2021-W08,2021-W09,2021-W10,2021-W11,2021-W12,2021-W13,2021-W14,2021-W15,2021-W16,2021-W17,2021-W18,2021-W19,2021-W20,2021-W21,2021-W22,2021-W23,2021-W24,2021-W25,2021-W26,2021-W27,2021-W28,2021-W29,2021-W30,2021-W31,2021-W32,2021-W33,2021-W34,2021-W35,2021-W36,2021-W37,2021-W38,2021-W39,2021-W40,2021-W41,2021-W42,2021-W43,2021-W44,2021-W45,2021-W46,2021-W47,2021-W48,2021-W49,2021-W50,2021-W51,2021-W52,2022-W01,2022-W02,2022-W03,2022-W04,2022-W05,2022-W06,2022-W07,2022-W08,2022-W09,2022-W10,2022-W11,2022-W12,2022-W13,2022-W14,2022-W15,2022-W16,2022-W17,2022-W18,2022-W19,2022-W20,2022-W21,2022-W22,2022-W23,2022-W24,2022-W25,2022-W26,2022-W27,2022-W28,2022-W29,2022-W30,2022-W31,2022-W32,2022-W33,2022-W34,2022-W35,2022-W36,2022-W37,2022-W38,2022-W39,2022-W40,2022-W41,2022-W42,2022-W43,2022-W44,2022-W45,2022-W46,2022-W47,2022-W48,2022-W49,2022-W50,2022-W51,2022-W52,2023-W01,2023-W02,2023-W03,2023-W04,2023-W05,2023-W06,2023-W07,2023-W08,2023-W09,2023-W10,2023-W11,2023-W12,2023-W13,2023-W14,2023-W15,2023-W16,2023-W17,2023-W18,2023-W19,2023-W20,2023-W21,2023-W22,2023-W23,2

In [462]:
# This code processes the raw death data (dd) by restructuring it into a long-form dataframe. 
# Each row in the new dataframe represents a single country's deaths for a specific year and week, 
# along with additional metadata such as country abbreviations and derived values.

temp = dd.melt(id_vars=['name','abbr','agegrp','agegrp_desc'],var_name='year-week',value_name='deaths')
temp['year'] = pd.to_numeric(temp['year-week'].str[0:4])
temp['week'] = pd.to_numeric(temp['year-week'].str[6:8])
temp['year.week'] = temp['year'] + temp['week']/100
temp['year.p'] = temp['year'] + (temp['week']/53.001)
dd = temp



In [463]:
# lets remove some data we don't need

# this is a combination of 27 countries
dd = dd[dd['abbr']!= 'EU27_2020'] 


dd = dd[dd['abbr']!= 'not available']
dd = dd[dd['abbr']!= 'Special value']
dd = dd[dd['abbr']!= 'None']
dd = dd[dd['abbr']!= 'Observation flags:']
dd = dd[dd['abbr']!= 'p']

# too early for 2025
dd = dd[dd['year']!= 2025]

dd = dd[~dd['name'].isna()]
dd = dd[~dd['agegrp'].isna()]

# we don't need these columns 
dd.drop(columns=['agegrp_desc'], inplace=True)
dd.drop(columns=['year-week'], inplace=True)

# converting columns 
dd['deaths'] = pd.to_numeric(dd['deaths'])

In [464]:
# there are quite a few NAN (not a number) values
# here we get rid of them 

# Get counts of NA values for each 'abbr' group
na_counts = dd[dd.deaths.isna()].groupby(['abbr','agegrp']).size()

# Filter for 'abbr' groups with more than 12 NA values
filtered_abbrs = na_counts[na_counts > 12*5].index

for abbr, agegrp in filtered_abbrs:
    print(f'removing -- {abbr} {abbr_to_name(abbr)} {agegrp} NACount={na_counts[(abbr, agegrp)]}')
    # Corrected filtering condition: remove rows where abbr and agegrp match separately
    dd = dd[~((dd.abbr == abbr) & (dd.agegrp == agegrp))]  # Use bitwise AND & inside the negation
    # print(len(dd))

removing -- AD Andorra TOTAL NACount=261
removing -- AD Andorra Y20-39 NACount=261
removing -- AD Andorra Y40-59 NACount=261
removing -- AD Andorra Y60-79 NACount=261
removing -- AD Andorra Y_GE80 NACount=261
removing -- AD Andorra Y_LT20 NACount=261
removing -- AL Albania TOTAL NACount=171
removing -- AL Albania Y20-39 NACount=171
removing -- AL Albania Y40-59 NACount=171
removing -- AL Albania Y60-79 NACount=171
removing -- AL Albania Y_GE80 NACount=171
removing -- AL Albania Y_LT20 NACount=171
removing -- GE Georgia TOTAL NACount=208
removing -- GE Georgia Y20-39 NACount=208
removing -- GE Georgia Y40-59 NACount=208
removing -- GE Georgia Y60-79 NACount=208
removing -- GE Georgia Y_GE80 NACount=208
removing -- GE Georgia Y_LT20 NACount=208
removing -- IE Ireland Y20-39 NACount=522
removing -- IE Ireland Y40-59 NACount=522
removing -- IE Ireland Y60-79 NACount=522
removing -- IE Ireland Y_GE80 NACount=522
removing -- IE Ireland Y_LT20 NACount=522
removing -- UK United Kingdom TOTAL N

In [465]:
# Fill missing values (NaNs) in the 2024 data with the mean deaths for 2024 by country

# Filter data for the year 2024
df_2024 = dd[dd.year == 2024]

# Group the 2024 data by country abbreviation ('abbr') and calculate summary statistics
grouped = df_2024.groupby(['abbr','agegrp'])

# Compute statistics: mean, total count, number of NaNs, total deaths, and mean deaths per week
stats = grouped['deaths'].agg(
    Mean='mean',                        # Average deaths for each country in 2024
    Count='size',                       # Total number of records for each country in 2024
    NACount=lambda x: x.isna().sum(),   # Count of missing (NaN) values
    SumFor2024='sum'                    # Total deaths for each country in 2024
).reset_index()

# Replace missing death values in 2024 with the calculated mean for each country
for _, row in stats.iterrows():
    dd.loc[
        (dd.abbr == row['abbr']) & (dd.agegrp == row['agegrp']) & (dd.year == 2024) & (dd.deaths.isna()), 
        'deaths'
    ] = row['Mean']

In [466]:

# we are going to create a new age group, everyone less than 80
# LT80 is less than 80 years old

# exclude where the age is greater than 80
temp = dd.copy()
temp = temp[temp['agegrp'] != 'TOTAL']
temp = temp[temp['agegrp'] != 'Y_GE80']

temp = pd.pivot_table(
    temp, 
    values='deaths', 
    index=['name', 'abbr','year','week','year.week','year.p'], 
    aggfunc='sum'
)
temp = temp.reset_index()

temp['agegrp'] = 'LT80'

dd = pd.concat([dd,temp])

In [467]:
# checking the data
display(dd.head(5))
display(dd.tail(5))

,name,abbr,agegrp,deaths,year,week,year.week,year.p
5,Belgium,BE,TOTAL,2461.0,2015,1,2015.01,2015.018868
6,Belgium,BE,Y_LT20,19.0,2015,1,2015.01,2015.018868
7,Belgium,BE,Y20-39,34.0,2015,1,2015.01,2015.018868
8,Belgium,BE,Y40-59,191.0,2015,1,2015.01,2015.018868
9,Belgium,BE,Y60-79,756.0,2015,1,2015.01,2015.018868


,name,abbr,agegrp,deaths,year,week,year.week,year.p
17221,Switzerland,CH,LT80,479.619048,2024,48,2024.48,2024.905643
17222,Switzerland,CH,LT80,479.619048,2024,49,2024.49,2024.924511
17223,Switzerland,CH,LT80,479.619048,2024,50,2024.50,2024.943378
17224,Switzerland,CH,LT80,479.619048,2024,51,2024.51,2024.962246
17225,Switzerland,CH,LT80,479.619048,2024,52,2024.52,2024.981114


In [468]:
# save to out folder
dd.to_csv(r'out\death_data.csv',index=False)

In [469]:

ddy = pd.pivot_table(
    dd,
    values='deaths',
    index=['name', 'abbr','agegrp','year'], 
    aggfunc='sum'
)
ddy = ddy.reset_index()

display(ddy.head(5))
display(ddy.tail(5))

# save to out folder
ddy.to_csv(r'out\death_data_year.csv',index=False)

,name,abbr,agegrp,year,deaths
0,Armenia,AM,LT80,2015,17785.0
1,Armenia,AM,LT80,2016,17334.0
2,Armenia,AM,LT80,2017,16426.0
3,Armenia,AM,LT80,2018,15379.0
4,Armenia,AM,LT80,2019,14880.0


,name,abbr,agegrp,year,deaths
2315,Switzerland,CH,Y_LT20,2020,491.000000
2316,Switzerland,CH,Y_LT20,2021,468.000000
2317,Switzerland,CH,Y_LT20,2022,511.000000
2318,Switzerland,CH,Y_LT20,2023,451.000000
2319,Switzerland,CH,Y_LT20,2024,407.333333


In [470]:
# here we normalize the death values

ddn = ddy.copy()
ddn['deaths_norm'] = 0.0

## baseline years are 2015,2016,2017 before the pandemic 
blyears = ddn[ddn.year.isin([2015,2016,2017])]
grouped = blyears.groupby(['abbr','agegrp'])

# Compute baseline mean
temp = grouped['deaths'].agg(
    baseline='mean',
).reset_index()

# Merge baseline means with the original DataFrame
ddn = ddn.merge(temp, on=['abbr', 'agegrp'], how='left')

# Normalize deaths column
ddn['deaths_norm'] = ddn['deaths'] / ddn['baseline']

# Drop the intermediate baseline column if not needed
ddn.drop(columns=['baseline'], inplace=True)

# save to out folder
ddn.to_csv(r'out\death_data_norm.csv',index=False)

### cause of death 

#### Getting the Data
1. go to [Europa.eu - Database](https://ec.europa.eu/eurostat/databrowser/explore/all/popul)
2. choose 
    * Population and social conditions 
    * Health
    * Causes of death
    * General mortality
    * Causes of death - deaths by country of residence and occurrence
3. Click the little table 󰓫
4. customize the data 
    * Customize your dataset -> Time -> From - to 
        * From: 2015
        * To: [Current or Max]
    * Customize your dataset -> `International Statistical Classification of Diseases and Related Health Problems (ICD-10 2010)`
        * All
            * UnCheck All
        * Aggregates
            * All Checked
    * Move the ``International Statistical Classification...` under `Geopolitical entity (reporting)`
5. Click `download` (as a spreadsheet) and place the file in the `.\data` folder


<img src='./docs/chrome_Gmc2itqQye.png' height=200/>

#### variables
* cod = cause of death data
* codn = cod, normalized


In [476]:
# getting the data 
cod = pd.read_excel(os.path.join(DataDIR,"hlth_cd_aro__custom_15177032_page_spreadsheet.xlsx"),sheet_name = "Sheet 1")

# remove the headers
cod = cod.iloc[9::]

# drop the bad columns
for c in cod.columns:
    if pd.isnull(cod.at[9,c]):
        cod = cod.drop(columns=[c])

# rename time columns
for c in cod.columns:
    name = cod.at[9,c]
    cod = cod.rename(columns={c: name})

# make the duplicate column names unique
cod = df_column_uniquify(cod)

# # rename the first two columns
cod = cod.rename(columns={'TIME': 'name'})
cod = cod.rename(columns={'TIME_1':'cod'})
           
# drop, replace, reset index,
cod = cod.drop([9,10])
cod = cod.replace(to_replace=':', value=None)
cod = cod.reset_index(drop=True)

cod['abbr'] = cod['name'].apply(name_to_abbr)

# display(cod.head(5))

c:\Users\JGarza\pythons\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [477]:
# This code processes the raw cod data (cod) by restructuring it into a long-form dataframe. 
# along with additional metadata such as country abbreviations and derived values.

temp = cod.melt(id_vars=['name','abbr','cod'],var_name='year',value_name='deaths')
temp['year'] = pd.to_numeric(temp['year'])
cod = temp

# display(cod.head(10))

In [478]:
# lets remove some data we don't need

# this is a combination of 27 countries
cod.loc[cod['name']== 'European Union - 27 countries (from 2020)','abbr'] = 'Euro27'


cod = cod[cod['name']!= 'not available']
cod = cod[cod['name']!= 'Special value']
cod = cod[cod['name']!= 'None']
cod = cod[cod['name']!= 'Observation flags:']
cod = cod[cod['name']!= 'p']
cod = cod[cod['name']!= 'Nan']
cod = cod[cod['name']!= 'd']

cod = cod[~cod['name'].isna()]

# display(cod.tail(10))

In [479]:
# there are quite a few NAN (not a number) values
# here we get rid of them 

# Get counts of NA values for each 'name' group
na_counts = cod[cod.deaths.isna()].groupby(['name','cod']).size()

# Filter for 'name' groups with more than 4 NA values
filtered_names = na_counts[na_counts > 4].index

# display(filtered_names)

for name, cod_val in filtered_names:
    print(f'removing -- {name} {abbr_to_name(name)} {cod_val} NACount={na_counts[(name, cod_val)]}')
    # Corrected filtering condition:
    cod = cod[~((cod['name'] == name) & (cod['cod'] == cod_val))]
    # print(len(cod))

removing -- Austria Abbreviation not found COVID-19, other NACount=5
19971
removing -- Belgium Abbreviation not found COVID-19, other NACount=6
19962
removing -- Bulgaria Abbreviation not found COVID-19, other NACount=6
19953
removing -- Croatia Abbreviation not found COVID-19, other NACount=6
19944
removing -- Cyprus Abbreviation not found COVID-19, other NACount=6
19935
removing -- Czechia Abbreviation not found COVID-19, other NACount=5
19926
removing -- Denmark Abbreviation not found COVID-19, other NACount=6
19917
removing -- Estonia Abbreviation not found COVID-19, other NACount=6
19908
removing -- European Union - 27 countries (from 2020) Abbreviation not found COVID-19, other NACount=7
19899
removing -- European Union - 28 countries (2013-2020) Abbreviation not found Accidental drowning and submersion NACount=5
19890
removing -- European Union - 28 countries (2013-2020) Abbreviation not found Accidental poisoning by and exposure to noxious substances NACount=5
19881
removing --

In [480]:
# save to out folder
cod.to_csv(r'out\cod_data.csv',index=False)

In [482]:
# here we normalize the cod data 

codn = cod.copy()
codn['deaths_norm'] = np.nan

## baseline years are 2015,2016,2017 before the pandemic 
blyears = codn[codn.year.isin([2015,2016,2017])]
grouped = blyears.groupby(['name','cod'])

# Compute baseline mean
temp = grouped['deaths'].agg(
    baseline='mean',
).reset_index()

# Merge baseline means with the original DataFrame
codn = codn.merge(temp, on=['name', 'cod'], how='left')

# display(codn.head(10))
# display(codn[codn['baseline'] == 0.0])
# display(codn[codn['baseline'].isna()])
# display(codn[codn['deaths'].isna()])

# # Normalize deaths column
# codn['deaths_norm'] = codn['deaths'] / codn['baseline']

codn['baseline'] = codn['baseline'].replace(0, np.nan)

codn.loc[(~codn['baseline'].isna()) & (~codn['deaths'].isna()), 'deaths_norm'] = codn['deaths'] / codn['baseline']

# Drop the intermediate baseline column if not needed
codn.drop(columns=['baseline'], inplace=True)

# # save to out folder
codn.to_csv(r'out\cod_data_norm.csv',index=False)

# display(codn.head(1000))


C:\Users\JGarza\AppData\Local\Temp\ipykernel_25668\1741213776.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  codn['baseline'] = codn['baseline'].replace(0, np.nan)
C:\Users\JGarza\AppData\Local\Temp\ipykernel_25668\1741213776.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.002917048087177 1.0491669305219198 1.0639549978364344 ... 0.5 0.3 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  codn.loc[(~codn['baseline'].isna()) & (~codn['deaths'].isna()), 'deaths_norm'] = codn['deaths'] / codn['baseline']


### vaccine data 

#### Getting the Data
1. [Data Download](https://www.ecdc.europa.eu/en/publications-data/data-covid-19-vaccination-eu-eea): Download the dataset in CSV format.

### combine all data

#### Observation
the charts below show the normalized deaths (by year) for each of the european countries in our dataset.

**A few things we could notice**
* all the countries were affected by the 2020 Covid-19 Pandemic 
    * some countries like Malta,Iceland,Netherlands has many years of higher than normal death rates
    * and others only had one or two bad years like Bulgaria and Romania
* even when viewing only agegrp LT80 (less than 80 years old), we can still see this pattern

In [338]:
title = 'Line Chart - normalized deaths in Europe 2015-2024 (by year)'
display(MD(f'### {title}'))

fig = px.line(
    ddn[ddn.agegrp == 'TOTAL'],
    x='year',
    y='deaths_norm',
    color='name',
    height=750 ,
    hover_data={
        'name', 'abbr', 'deaths_norm'
    },
    title=title
    )
fig.update_layout(template="plotly_dark")

### Line Chart - normalized deaths in Europe 2015-2024 (by year)

In [339]:
## this will order by high to low deaths after 2023 (pre-pandemic)
sums = pd.pivot_table(
    ddn[ddn.agegrp == 'TOTAL'][ddn.year >= 2023], # totals only 
    values = 'deaths_norm',
    index = ['abbr','name'],
    aggfunc='sum',
)
sums = sums.reset_index()
sums = sums.sort_values(by='deaths_norm',ascending=False)
# display(sums)

display(MD('### HeatMap (based on Year)'))
temp = pd.pivot_table(
    ddn[ddn.agegrp == 'TOTAL'], # totals only 
    values = 'deaths_norm',
    index = ['abbr','name'],
    columns=['year'],
    aggfunc='mean',
)
temp.columns.name = 'index' 

# temp = temp.sort_values(by=2024,ascending=False)
temp = temp.reset_index()
temp = temp.set_index('name').reindex(sums['name']).reset_index()

display(temp.style.background_gradient(cmap=heatmapCM,axis=1))

C:\Users\JGarza\AppData\Local\Temp\ipykernel_30076\151973392.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



### HeatMap (based on Year)

index,name,abbr,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Malta,MT,1.016949,0.955701,1.027350,1.060863,1.064330,1.199538,1.196071,1.219183,1.161980,1.184373
1,Iceland,IS,0.988146,1.020151,0.991702,1.000148,1.005927,1.042377,1.039265,1.195733,1.140169,1.156252
2,Netherlands,NL,1.003373,0.993167,1.003460,1.024649,1.015107,1.147749,1.140424,1.133556,1.131250,1.143136
3,Cyprus,CY,1.029186,0.937156,1.033658,0.997362,1.069610,1.143922,1.241800,1.239908,1.150459,1.112573
4,Ireland,IE,1.007002,0.998439,0.994559,1.013608,1.002031,1.041412,1.098655,1.128063,1.125593,1.106379
5,Finland,FI,0.996378,1.003504,1.000118,1.016952,1.004682,1.051873,1.076956,1.155757,1.143393,1.064547
6,Germany,DE,1.016481,0.979311,1.004209,1.029001,1.012227,1.082065,1.101127,1.149954,1.106922,1.074998
7,Luxembourg,LU,0.992076,0.966098,1.041827,1.053835,1.050895,1.143779,1.100400,1.089127,1.081529,1.080324
8,Denmark,DK,1.007454,0.991352,1.001194,1.038393,1.014429,1.045972,1.075440,1.117541,1.090769,1.064901
9,Austria,AT,1.023795,0.972870,1.003336,1.012615,1.006371,1.125312,1.104853,1.130013,1.081778,1.062992


In [344]:
## this will order by high to low deaths after 2023 (pre-pandemic)
sums = pd.pivot_table(
    ddn[ddn.agegrp == 'LT80'][ddn.year >= 2023], # totals only 
    values = 'deaths_norm',
    index = ['abbr','name'],
    aggfunc='sum',
)
sums = sums.reset_index()
sums = sums.sort_values(by='deaths_norm',ascending=False)
# display(sums)

display(MD('### HeatMap (based on Year, only where agegrp is LT80)'))
temp = pd.pivot_table(
    ddn[ddn.agegrp == 'LT80'], # totals only 
    values = 'deaths_norm',
    index = ['abbr','name'],
    columns=['year'],
    aggfunc='mean',
)
temp.columns.name = 'index' 

# temp = temp.sort_values(by=2024,ascending=False)
temp = temp.reset_index()
temp = temp.set_index('name').reindex(sums['name']).reset_index()

display(temp.style.background_gradient(cmap=heatmapCM,axis=1))

C:\Users\JGarza\AppData\Local\Temp\ipykernel_30076\3268976977.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



### HeatMap (based on Year, only where agegrp is LT80)

index,name,abbr,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Iceland,IS,0.963504,1.036496,1.000000,1.019812,1.012513,1.105318,1.123045,1.273201,1.211679,1.256991
1,Malta,MT,1.017293,0.967203,1.015504,1.057841,1.036971,1.166369,1.196780,1.207513,1.124031,1.158677
2,Netherlands,NL,1.009040,0.998123,0.992837,1.013964,1.004492,1.114691,1.143528,1.122958,1.113380,1.110753
3,Norway,NO,1.003819,0.998207,0.997974,1.013113,1.015627,1.033864,1.050348,1.137911,1.107398,1.106405
4,Cyprus,CY,1.026756,0.958661,1.014583,1.020289,1.053005,1.140122,1.280878,1.213923,1.143546,1.059914
5,France,FR,1.012897,0.989268,0.997835,1.006304,1.003646,1.094593,1.112343,1.124722,1.086984,1.075147
6,Spain,ES,1.029906,0.983827,0.986268,0.997999,0.994437,1.164643,1.109420,1.114142,1.082376,1.065384
7,Finland,FI,1.000246,1.000328,0.999426,0.998402,0.988075,1.026145,1.040611,1.090771,1.100033,1.044840
8,Poland,PL,1.017801,0.981581,1.000618,1.023203,1.018394,1.181092,1.309963,1.115682,1.035903,1.025043
9,Luxembourg,LU,0.979849,0.945681,1.074470,1.028211,1.006658,1.077098,1.061854,1.024006,1.026108,1.027335


## Conclusion